In [3]:
!pip install kaggle

In [4]:
import kaggle
import pandas as pd

In [5]:
!kaggle datasets download -d dheemanthbhat/stack-overflow-annual-developer-survey-2022

^C


In [32]:
df = pd.read_csv('survey_results_public.csv')

In [33]:
df = df.drop_duplicates()

In [34]:
df.YearsCodePro.dtype

dtype('O')

In [35]:
df.CompTotal.dtype

dtype('float64')

In [88]:
# This shows the difference between women and men sallary
# I just used a minimum of 10000 and maximum 1 million and I picked only sallary in USD
# And there were some problem if I try to compare currency to 'USD United States dollar' so again instead of cleaning data
# I just look at first 3 letters and check if it's 'USD'
# I picked only full-time employers to make results fair

women_salary = 0
women_count = 0
men_salary = 0
men_count = 0

for i in range(len(df['Gender'])):
    if df['Gender'][i] == 'Woman' and 10000 < df['ConvertedCompYearly'][i] < 1000000 and df['ConvertedCompYearly'][i] != 'NaN' and df['Currency'][i][:3] == 'USD' and df['Employment'][i][:19] == 'Employed, full-time':
        women_salary += int(df['ConvertedCompYearly'][i])
        women_count += 1
        
    if df['Gender'][i] == 'Man' and 10000 < df['ConvertedCompYearly'][i] < 1000000 and df['ConvertedCompYearly'][i] != 'NaN' and df['Currency'][i][:3] == 'USD' and df['Employment'][i][:19] == 'Employed, full-time':
        men_salary += int(df['ConvertedCompYearly'][i])
        men_count += 1
        
print(f"Women's average sallary: {int(women_salary/women_count)} USD, Men's average sallary {int(men_salary/men_count)} USD")

Women's average sallary: 142368 USD, Men's average sallary 152068 USD


In [37]:
print(len('Employed, full-time'))

19


In [90]:
inperson_salary, inperson_count = 0, 0
hybrid_salary, hybrid_count = 0, 0
remote_salary, remote_count = 0, 0

for i in range(len(df['ConvertedCompYearly'])):
    if df['Gender'][i] == 'Woman' and 10000 < df['ConvertedCompYearly'][i] < 1000000 and df['ConvertedCompYearly'][i] != 'NaN' and df['Currency'][i][:3] == 'USD' and df['Employment'][i][:19] == 'Employed, full-time' and df['RemoteWork'][i].split(' ')[1] == 'in-person':
        inperson_salary += int(df['ConvertedCompYearly'][i])
        inperson_count += 1
    if df['Gender'][i] == 'Woman' and 10000 < df['ConvertedCompYearly'][i] < 1000000 and df['ConvertedCompYearly'][i] != 'NaN' and df['Currency'][i][:3] == 'USD' and df['Employment'][i][:19] == 'Employed, full-time' and df['RemoteWork'][i].split(' ')[0] == 'Hybrid':
        hybrid_salary += int(df['ConvertedCompYearly'][i])
        hybrid_count += 1
    if df['Gender'][i] == 'Woman' and 10000 < df['ConvertedCompYearly'][i] < 1000000 and df['ConvertedCompYearly'][i] != 'NaN' and df['Currency'][i][:3] == 'USD' and df['Employment'][i][:19] == 'Employed, full-time' and df['RemoteWork'][i].split(' ')[1] == 'remote':
        remote_salary += int(df['ConvertedCompYearly'][i])
        remote_count += 1

print(f"Avg. in-person salary: {int(inperson_salary/inperson_count)}USD, Avg. hybrid salary: {int(hybrid_salary/hybrid_count)}USD, Avg. remote salary {int(remote_salary/remote_count)}USD")

Avg. in-person salary: 130479USD, Avg. hybrid salary: 139101USD, Avg. remote salary 144764USD


In [39]:
# what we see is that salary of people working full in-person is definitely lower than others
# but it's not so clear that working full in-person is less profitable than other possibilites
# because in a lot of companies, people with the lowest experience/interns have to work full in-person
# and people with higher experience can choose where they want to work from
# so to make it clear I will pick only people with 'YearsCodePro' higher than 4 years

In [40]:
values = set()
for value in df['YearsCodePro']:
    if value not in values:
        values.add(value)

print(values)

{'32', '5', '7', '13', '20', '23', '15', '19', '18', '38', '31', 'More than 50 years', '39', '12', '34', '16', '40', '45', '27', '24', '44', '29', '1', '37', '41', '3', '6', '14', '4', '46', '9', '30', '43', '21', '47', '17', '35', '33', nan, '8', '22', '25', '50', '42', '49', '2', '28', '26', 'Less than 1 year', '10', '11', '48', '36'}


In [52]:
# clean values for the purpose of this task, I change 'less than 1 year' into '1' and 'more than 50 years' into '50'
# which is enough for me
df['YearsCodePro'] = df['YearsCodePro'].str.replace('[^0-9]','', regex=True)

In [65]:
values = set()
for value in df['YearsCodePro']:
    if value not in values and value != "NaN":
        values.add(value)

print(values)

{'32', '5', '7', '13', '20', '23', '15', '19', '18', '38', '31', '39', '12', '34', '16', '40', '45', '27', '24', '44', '29', '1', '37', '41', '3', '6', '14', '4', '46', '9', '30', '43', '21', '47', '17', '35', '33', nan, '8', '22', '25', '50', '42', '49', '2', '28', '26', '10', '11', '48', '36'}


In [97]:
inperson_salary, inperson_count, inperson_years_code_pro = 0, 0, 0
hybrid_salary, hybrid_count, hybrid_years_code_pro = 0, 0, 0
remote_salary, remote_count, remote_years_code_pro = 0, 0, 0

for i in range(len(df['ConvertedCompYearly'])):
    if 10000 < df['ConvertedCompYearly'][i] < 1000000 and df['ConvertedCompYearly'][i] != 'NaN' and df['Currency'][i][:3] == 'USD' and df['Employment'][i][:19] == 'Employed, full-time' and df['RemoteWork'][i].split(' ')[1] == 'in-person' and float(df['YearsCodePro'][i]) > 4:
        inperson_salary += int(df['CompTotal'][i])
        inperson_count += 1
        inperson_years_code_pro += int(df['YearsCodePro'][i])
            
    if 10000 < df['ConvertedCompYearly'][i] < 1000000 and df['ConvertedCompYearly'][i] != 'NaN' and df['Currency'][i][:3] == 'USD' and df['Employment'][i][:19] == 'Employed, full-time' and df['RemoteWork'][i].split(' ')[0] == 'Hybrid' and float(df['YearsCodePro'][i]) > 4:
        hybrid_salary += int(df['CompTotal'][i])
        hybrid_count += 1
        hybrid_years_code_pro += int(df['YearsCodePro'][i])
            
    if 10000 < df['ConvertedCompYearly'][i] < 1000000 and df['ConvertedCompYearly'][i] != 'NaN' and df['Currency'][i][:3] == 'USD' and df['Employment'][i][:19] == 'Employed, full-time' and df['RemoteWork'][i].split(' ')[1] == 'remote' and float(df['YearsCodePro'][i]) > 4:
        remote_salary += int(df['ConvertedCompYearly'][i])
        remote_count += 1
        remote_years_code_pro += int(df['YearsCodePro'][i])

avg_inperson_salary = int(inperson_salary/inperson_count)        
avg_hybrid_salary = int(hybrid_salary/hybrid_count)
avg_remote_salary = int(remote_salary/remote_count)

avg_inperson_years_code_pro = round(inperson_years_code_pro/inperson_count, 2)
avg_hybrid_years_code_pro = round(hybrid_years_code_pro/hybrid_count, 2)
avg_remote_years_code_pro = round(remote_years_code_pro/remote_count, 2)

print(f"Avg. in-person salary: {avg_inperson_salary}USD and avg in-person years of coding as a pro: {avg_inperson_years_code_pro} years\n"
        f"Avg. hybrid salary: {avg_hybrid_salary}USD and avg hybrid years of coding as a pro: {avg_hybrid_years_code_pro} years\n"
        f"Avg. remote salary {avg_remote_salary}USD and avg remote years of coding as a pro: {avg_remote_years_code_pro} years")

Avg. in-person salary: 135406USD and avg in-person years of coding as a pro: 14.53 years
Avg. hybrid salary: 165517USD and avg hybrid years of coding as a pro: 13.86 years
Avg. remote salary 162372USD and avg remote years of coding as a pro: 14.23 years


In [96]:
# it shows that people who work hybrid or fully remote earn over 20% more than people who work in-person
# despite the fact that on average in-person workers code as a pro longer than the others